## Familiarization with the data

Imports and the libraries I will be using and data reading:

“timeStamp”: is the date time of the mesure.

“Idflowmeter”: is the ID of the meter The peculiar series of water measures for two sensor IDs are yet to be explained by Braila.

“Tot1”: is the water flow in m3

“Tot2”: is the water flow in m3. The difference between Tot1 and Tot2 are yet to be explained by Braila.

“Analog2”: is the water pressure of the measure point. The unit is probably Pascal, however not yet confirmed by Braila.

In [1]:
import math
import pandas as pandas
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pandas.read_json("../data/water_loss_data_set_1.json")

# removing false values
data = data[(data.timeStamp != "0000-00-00 00:00:00" ) & (data.timeStamp != "2000-01-01 00:00:00")]

Processing...
1. find min and max of Tot1, Tot2 and Analog2.

Removed this timestamps from data  
`['0000-00-00 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00']`

In [3]:
data

,timeStamp,idflowmeter,tot1,tot2,analog2
0,2018-11-23 07:03:00,MAG8000_024905H318,0.00,0.00,0.00
1,2018-11-23 07:04:00,MAG8000_024905H318,0.00,0.00,0.00
2,2018-11-23 07:06:00,MAG8000_024905H318,0.00,0.00,0.00
3,2018-11-23 07:09:00,MAG8000_024905H318,0.00,0.00,0.00
4,2018-11-23 07:10:00,MAG8000_024905H318,0.00,0.00,0.00
...,...,...,...,...,...
1395740,2020-04-01 14:05:00,MAG8000_024805H318,977428.25,-4713.21,1.54
1395741,2020-04-01 14:06:00,MAG8000_024805H318,977428.88,-4713.21,1.54
1395742,2020-04-01 14:07:00,MAG8000_024805H318,977429.50,-4713.21,1.54
1395743,2020-04-01 14:08:00,MAG8000_024805H318,977430.12,-4713.21,1.54


In [6]:
test_data = data[["timeStamp","tot1"]]
test_array = test_data.to_numpy()
# sorting by time stamp
test_array = np.sort(test_array, axis=0)

timestamp = [i[0] for i in test_array]
tot1 = [i[1] for i in test_array]
print("\ntime :\n", timestamp[:10], "\ntot1 :\n", tot1[:10])
print("")
print("Max date:", max(timestamp), "\nMin date:", min(timestamp), "\nMedian date:", timestamp[round(len(timestamp)/2)])
#print("\n\nMax date:", timestamp[len(timestamp)-1], "\nMin date:", timestamp[0], "\nMedian date:", 
#      timestamp[round(len(timestamp)/2)])
#print("\n\nFalse values: ", test_array[:10])


time :
 ['2018-11-23 07:03:00', '2018-11-23 07:04:00', '2018-11-23 07:06:00', '2018-11-23 07:09:00', '2018-11-23 07:10:00', '2018-11-23 07:11:00', '2018-11-23 07:12:00', '2018-11-23 07:13:00', '2018-11-23 07:14:00', '2018-11-23 07:15:00'] 
tot1 :
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Max date: 2020-04-01 14:09:00 
Min date: 2018-11-23 07:03:00 
Median date: 2019-08-02 02:28:00


time :  
 ['0000-00-00 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2000-01-01 00:00:00', '2018-11-23 07:03:00', '2018-11-23 07:04:00', '2018-11-23 07:06:00', '2018-11-23 07:09:00', '2018-11-23 07:10:00'] 
tot1 :  
 [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Max date: 2020-04-01 14:09:00   
Min date: 0000-00-00 00:00:00 
Median date: 2019-08-02 02:27:00  


Max date: 2020-04-01 14:09:00 
Min date: 0000-00-00 00:00:00 
Median date: 2019-08-02 02:27:00


False values:  [['0000-00-00 00:00:00' -1.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2000-01-01 00:00:00' 0.0]
 ['2018-11-23 07:03:00' 0.0]
 ['2018-11-23 07:04:00' 0.0]
 ['2018-11-23 07:06:00' 0.0]
 ['2018-11-23 07:09:00' 0.0]
 ['2018-11-23 07:10:00' 0.0]]

In [7]:
"""fig = plt.figure(figsize=(12, 8), dpi= 100, facecolor='w')
plt.plot(timestamp, tot1)
#plt.ylabel('Število ogledov/ocen')
# plt.xlabel('Povprečna ocena')
plt.title('Time')
plt.show()""";